In [27]:
import pathlib
import requests
import re
import pandas as pd
import matplotlib as mpl
import matplotlib.pyplot as plt

In [28]:
from requests.packages.urllib3.exceptions import InsecureRequestWarning
requests.packages.urllib3.disable_warnings(InsecureRequestWarning)

In [14]:
# cwd = pathlib.Path.cwd()
# cwd
docs_dir = pathlib.Path("/private/var/mobile/Library/Mobile Documents/iCloud~AsheKube~Carnets/Documents")
data_dir = docs_dir / "data"
if not data_dir.is_dir():
    data_dir.mkdir()
    print(">>> created dir {}".format(data_dir))

In [29]:
# base_tsdata_url = "https://github.com/CSSEGISandData/COVID-19/blob/master/csse_covid_19_data"
base_tsdata_url = "https://github.com/CSSEGISandData/COVID-19/raw/master/csse_covid_19_data"
deaths_global_url = "{}/csse_covid_19_time_series/time_series_covid19_deaths_global.csv".format(base_tsdata_url)
deaths_global_req = requests.get(deaths_global_url)
deaths_global_raw = deaths_global_req.content
deaths_global_csv = data_dir / "time_series_covid19_deaths_global.csv"
with deaths_global_csv.open('w') as f:
    f.write(deaths_global_raw.decode())
deaths_global = pd.read_csv(deaths_global_csv)
deaths_global

Province/State          Country/Region        Lat  \
0                             NaN             Afghanistan  33.000000   
1                             NaN                 Albania  41.153300   
2                             NaN                 Algeria  28.033900   
3                             NaN                 Andorra  42.506300   
4                             NaN                  Angola -11.202700   
5                             NaN     Antigua and Barbuda  17.060800   
6                             NaN               Argentina -38.416100   
7                             NaN                 Armenia  40.069100   
8    Australian Capital Territory               Australia -35.473500   
9                 New South Wales               Australia -33.868800   
10             Northern Territory               Australia -12.463400   
11                     Queensland               Australia -28.016700   
12                South Australia               Australia -34.928500   
13                       Tasmania               Australia -41.454500   
14                       Victoria               Australia -37.813600   
15              Western Australia               Australia -31.950500   
16                            NaN                 Austria  47.516200   
17                            NaN              Azerbaijan  40.143100   
18                            NaN                 Bahamas  25.034300   
19                            NaN                 Bahrain  26.027500   
20                            NaN              Bangladesh  23.685000   
21                            NaN                Barbados  13.193900   
22                            NaN                 Belarus  53.709800   
23                            NaN                 Belgium  50.833300   
24                            NaN                   Benin   9.307700   
25                            NaN                  Bhutan  27.514200   
26                            NaN                 Bolivia -16.290200   
27                            NaN  Bosnia and Herzegovina  43.915900   
28                            NaN                  Brazil -14.235000   
29                            NaN                  Brunei   4.535300   
..                            ...                     ...        ...   
224                           NaN                 Uruguay -32.522800   
225                           NaN                      US  37.090200   
226                           NaN              Uzbekistan  41.377500   
227                           NaN               Venezuela   6.423800   
228                           NaN                 Vietnam  16.000000   
229                           NaN                  Zambia -15.416700   
230                           NaN                Zimbabwe -20.000000   
231              Diamond Princess                  Canada   0.000000   
232                           NaN                Dominica  15.415000   
233                           NaN                 Grenada  12.116500   
234                           NaN              Mozambique -18.665695   
235                           NaN                   Syria  34.802075   
236                           NaN             Timor-Leste  -8.874217   
237                           NaN                  Belize  13.193900   
238                     Recovered                  Canada   0.000000   
239                           NaN                    Laos  19.856270   
240                           NaN                   Libya  26.335100   
241                           NaN      West Bank and Gaza  31.952200   
242                           NaN           Guinea-Bissau  11.803700   
243                           NaN                    Mali  17.570692   
244                           NaN   Saint Kitts and Nevis  17.357822   
245         Northwest Territories                  Canada  64.825500   
246                         Yukon                  Canada  64.282300   
247                           NaN                  Kosovo  

In [16]:
print(deaths_global.shape)
# deaths_global.transpose()
# deaths_global.columns
# deaths_global.columns[0] = 'subregion'
date_re = re.compile(r"\d+/\d+/\d+")
all_cols = list(deaths_global.columns)
date_cols = list(filter(date_re.match, all_cols))
indx_cols = [i for i in all_cols if i not in date_cols]
print(indx_cols)
print(date_cols)
locids = deaths_global.index


(254, 73)
['Province/State', 'Country/Region', 'Lat', 'Long']
['1/22/20', '1/23/20', '1/24/20', '1/25/20', '1/26/20', '1/27/20', '1/28/20', '1/29/20', '1/30/20', '1/31/20', '2/1/20', '2/2/20', '2/3/20', '2/4/20', '2/5/20', '2/6/20', '2/7/20', '2/8/20', '2/9/20', '2/10/20', '2/11/20', '2/12/20', '2/13/20', '2/14/20', '2/15/20', '2/16/20', '2/17/20', '2/18/20', '2/19/20', '2/20/20', '2/21/20', '2/22/20', '2/23/20', '2/24/20', '2/25/20', '2/26/20', '2/27/20', '2/28/20', '2/29/20', '3/1/20', '3/2/20', '3/3/20', '3/4/20', '3/5/20', '3/6/20', '3/7/20', '3/8/20', '3/9/20', '3/10/20', '3/11/20', '3/12/20', '3/13/20', '3/14/20', '3/15/20', '3/16/20', '3/17/20', '3/18/20', '3/19/20', '3/20/20', '3/21/20', '3/22/20', '3/23/20', '3/24/20', '3/25/20', '3/26/20', '3/27/20', '3/28/20', '3/29/20', '3/30/20']


In [17]:
# deaths_global[indx_cols]
# pd.MultiIndex([list(deaths_global['Province/State'])], names=['subregion'])
mindx_df = deaths_global[indx_cols]
mindx_df['locid'] = locids
mindx_df.columns = ['subregion', 'region', 'lat', 'long', 'locid']
mindx_df = mindx_df.iloc[:, [4, 1, 0, 2, 3]]
mindx = pd.MultiIndex.from_frame(mindx_df)
# mindx_df
deaths_global_BAK = deaths_global.loc[:, :]
deaths_global = deaths_global_BAK[date_cols].transpose()
deaths_global.columns = mindx
row_indx = deaths_global.index
deaths_global.index = pd.to_datetime(row_indx)
deaths_global

/var/mobile/Containers/Data/Application/D6D5B0B0-C121-4DA7-96BC-DAA124CB5F6F/Library/lib/python3.7/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  after removing the cwd from sys.path.


locid              0           1           2           3           4    \
region     Afghanistan     Albania     Algeria     Andorra      Angola   
subregion          NaN         NaN         NaN         NaN         NaN   
lat          33.000000   41.153300   28.033900   42.506300  -11.202700   
long        65.000000   20.168300   1.659600    1.521800    17.873900    
2020-01-22           0           0           0           0           0   
2020-01-23           0           0           0           0           0   
2020-01-24           0           0           0           0           0   
2020-01-25           0           0           0           0           0   
2020-01-26           0           0           0           0           0   
2020-01-27           0           0           0           0           0   
2020-01-28           0           0           0           0           0   
2020-01-29           0           0           0           0           0   
2020-01-30           0           0           0           0           0   
2020-01-31           0           0           0           0           0   
2020-02-01           0           0           0           0           0   
2020-02-02           0           0           0           0           0   
2020-02-03           0           0           0           0           0   
2020-02-04           0           0           0           0           0   
2020-02-05           0           0           0           0           0   
2020-02-06           0           0           0           0           0   
2020-02-07           0           0           0           0           0   
2020-02-08           0           0           0           0           0   
2020-02-09           0           0           0           0           0   
2020-02-10           0           0           0           0           0   
2020-02-11           0           0           0           0           0   
2020-02-12           0           0           0           0           0   
2020-02-13           0           0           0           0           0   
2020-02-14           0           0           0           0           0   
2020-02-15           0           0           0           0           0   
2020-02-16           0           0           0           0           0   
2020-02-17           0           0           0           0           0   
2020-02-18           0           0           0           0           0   
2020-02-19           0           0           0           0           0   
2020-02-20           0           0           0           0           0   
...                ...         ...         ...         ...         ...   
2020-03-01           0           0           0           0           0   
2020-03-02           0           0           0           0           0   
2020-03-03           0           0           0           0           0   
2020-03-04           0           0           0           0           0   
2020-03-05           0           0           0           0           0   
2020-03-06           0           0           0           0           0   
2020-03-07           0           0           0           0           0   
2020-03-08           0           0           0           0           0   
2020-03-09           0           0           0           0           0   
2020-03-10           0           0           0           0           0   
2020-03-11           0           1           0           0           0   
2020-03-12           0           1           1           0           0   
2020-03-13           0           1           2           0           0   
2020-03-14           0           1           3           0           0   
2020-03-15           0           1           4           0           0   
2020-03-16           0           1           4           0           0   
2020-03-17           0           1           4           0           0   
2020-03-18           0           2           7           0           0   
2020

In [20]:
deaths_global

locid              0           1           2           3           4    \
region     Afghanistan     Albania     Algeria     Andorra      Angola   
subregion          NaN         NaN         NaN         NaN         NaN   
lat          33.000000   41.153300   28.033900   42.506300  -11.202700   
long        65.000000   20.168300   1.659600    1.521800    17.873900    
2020-01-22           0           0           0           0           0   
2020-01-23           0           0           0           0           0   
2020-01-24           0           0           0           0           0   
2020-01-25           0           0           0           0           0   
2020-01-26           0           0           0           0           0   
2020-01-27           0           0           0           0           0   
2020-01-28           0           0           0           0           0   
2020-01-29           0           0           0           0           0   
2020-01-30           0           0           0           0           0   
2020-01-31           0           0           0           0           0   
2020-02-01           0           0           0           0           0   
2020-02-02           0           0           0           0           0   
2020-02-03           0           0           0           0           0   
2020-02-04           0           0           0           0           0   
2020-02-05           0           0           0           0           0   
2020-02-06           0           0           0           0           0   
2020-02-07           0           0           0           0           0   
2020-02-08           0           0           0           0           0   
2020-02-09           0           0           0           0           0   
2020-02-10           0           0           0           0           0   
2020-02-11           0           0           0           0           0   
2020-02-12           0           0           0           0           0   
2020-02-13           0           0           0           0           0   
2020-02-14           0           0           0           0           0   
2020-02-15           0           0           0           0           0   
2020-02-16           0           0           0           0           0   
2020-02-17           0           0           0           0           0   
2020-02-18           0           0           0           0           0   
2020-02-19           0           0           0           0           0   
2020-02-20           0           0           0           0           0   
...                ...         ...         ...         ...         ...   
2020-03-01           0           0           0           0           0   
2020-03-02           0           0           0           0           0   
2020-03-03           0           0           0           0           0   
2020-03-04           0           0           0           0           0   
2020-03-05           0           0           0           0           0   
2020-03-06           0           0           0           0           0   
2020-03-07           0           0           0           0           0   
2020-03-08           0           0           0           0           0   
2020-03-09           0           0           0           0           0   
2020-03-10           0           0           0           0           0   
2020-03-11           0           1           0           0           0   
2020-03-12           0           1           1           0           0   
2020-03-13           0           1           2           0           0   
2020-03-14           0           1           3           0           0   
2020-03-15           0           1           4           0           0   
2020-03-16           0           1           4           0           0   
2020-03-17           0           1           4           0           0   
2020-03-18           0           2           7           0           0   
2020

In [21]:
##### fignum = 0
fig = plt.figure(fignum)
plot_data = deaths_global.iloc[:, [0, 1, 2]]
# plot_data.columns.names
plot_regions = plot_data.columns[[0, 1, 2]].get_level_values('region')
plt.plot(plot_data)
plt.legend(plot_regions)

NameError: name 'fignum' is not defined

In [22]:
d = {}
tskeys = [
    'confirmed_US',
    'confirmed_global',
    'deaths_US',
    'deaths_global',
    'recovered_global'
]
for tsk in tskeys:
    d[tsk] = {}
    
d

{'confirmed_US': {},
 'confirmed_global': {},
 'deaths_US': {},
 'deaths_global': {},
 'recovered_global': {}}

In [36]:
# deaths_global_url = "{}/csse_covid_19_time_series/time_series_covid19_deaths_global.csv".format(base_tsdata_url)
# deaths_global_req = requests.get(deaths_global_url)
# deaths_global_raw = deaths_global_req.content
# deaths_global_csv = data_dir / "time_series_covid19_deaths_global.csv"
# with deaths_global_csv.open('w') as f:
#     f.write(deaths_global_raw.decode())
# deaths_global = pd.read_csv(deaths_global_csv)

for tsk in tskeys:
    print(">>> getting {}".format(tsk))
    d[tsk]['url'] = f"{base_tsdata_url}/csse_covid_19_time_series/time_series_covid19_{tsk}.csv"
    d[tsk]['req'] = requests.get(d[tsk]['url'], auth=('user', 'pass'))
    d[tsk]['raw'] = d[tsk]['req'].content
    d[tsk]['csv'] = data_dir / f"time_series_covid19_{tsk}.csv"
    with d[tsk]['csv'].open('w') as f:
        print("--> writing .../{} ... ".format(d[tsk]['csv'].name), end='')
        f.write(d[tsk]['raw'].decode())
        print("wrote {} B".format(d[tsk]['csv'].stat().st_size))
    d[tsk]['df'] = pd.read_csv(d[tsk]['csv'])
    print("--> read CSV data\n")
# d

>>> getting confirmed_US
--> writing .../time_series_covid19_confirmed_US.csv ... wrote 766891 B
--> read CSV data

>>> getting confirmed_global
--> writing .../time_series_covid19_confirmed_global.csv ... wrote 52577 B
--> read CSV data

>>> getting deaths_US
--> writing .../time_series_covid19_deaths_US.csv ... wrote 779130 B
--> read CSV data

>>> getting deaths_global
--> writing .../time_series_covid19_deaths_global.csv ... wrote 44110 B
--> read CSV data

>>> getting recovered_global
--> writing .../time_series_covid19_recovered_global.csv ... wrote 45034 B
--> read CSV data

